In [ ]:
import argparse
import copy
import os
import shlex

from IPython.display import display
from PIL import Image
import numpy as np

from learn_nerf.dataset import CameraView
from learn_nerf.scripts.render_nerf import RenderSession, argparser

In [ ]:
parser = argparser()
parser.add_argument('start_view', type=str)

# Example of some arguments.
arg_str = "--model llff_nerfs/nerf_v14.pkl --instant_ngp --width 256 --height 256 /media/dumpster1/colmap_test/room/nerf_dataset_v14/metadata.json /media/dumpster1/colmap_test/room/nerf_dataset_v14/00000.json"
args = parser.parse_args(shlex.split(arg_str))
renderer = RenderSession(args)

In [ ]:
LEFT_RIGHT_THETA = 0.2
UP_DOWN_THETA = 0.2
FORWARD_DIST = 0.3

saved_views = []
view = CameraView.from_json(args.start_view)

In [ ]:
def render_loop():
    while True:
        saved_views.append(copy.deepcopy(view))
        renderer.render_view(view)
        display(Image.fromarray(renderer.images[-1]))
        cmds = ''
        while not cmds or any(x not in list('rludfbxo') for x in cmds):
            cmds = input('r=right, l=left, u=up, d=down, f=forward, b=back, o=reorient, x=stop: ')
        for cmd in cmds:
            if cmd == 'x':
                return
            elif cmd == 'r' or cmd == 'l':
                th = -LEFT_RIGHT_THETA
                if cmd == 'l':
                    th = -th
                x, z = np.array(view.x_axis), np.array(view.camera_direction)
                view.x_axis = tuple(x*np.cos(th) + z*np.sin(th))
                view.camera_direction = tuple(-x*np.sin(th) + z*np.cos(th))
            elif cmd == 'u' or cmd == 'd':
                th = -UP_DOWN_THETA
                if cmd == 'u':
                    th = -th
                x, z = np.array(view.y_axis), np.array(view.camera_direction)
                view.y_axis = tuple(x*np.cos(th) + z*np.sin(th))
                view.camera_direction = tuple(-x*np.sin(th) + z*np.cos(th))
            elif cmd == 'f' or cmd == 'b':
                d = FORWARD_DIST
                if cmd == 'b':
                    d = -d
                view.camera_origin = tuple(np.array(view.camera_origin) + np.array(view.camera_direction)*d)
            elif cmd == 'o':
                x, y, z = np.array(view.x_axis), np.array(saved_views[0].y_axis), np.array(view.camera_direction)
                z = z - y*np.dot(z, y)
                z = z / np.linalg.norm(z)
                x = np.cross(y, z)
                x = x / np.linalg.norm(x)
                view.x_axis = tuple(x)
                view.y_axis = tuple(y)
                view.camera_direction = tuple(z)

render_loop()

In [ ]:
# Save a low-res version to a single reel file.
renderer.save('interactive.png')

In [ ]:
# Render in higher resolution for ffmpeg encoding.
os.makedirs('interactive', exist_ok=True)
renderer.images = []
args.width = 384
args.height = 384
for i, view in enumerate(saved_views):
    print(f'view {i} of {len(saved_views)}')
    renderer.render_view(view)
    Image.fromarray(renderer.images.pop()).save(f'interactive/{i:04}.png')